This script is the preliminary main operator script for running functions through the az-tilt estimation pipeline. To test out a function, we run the following steps:

1) Collect the data sets to test on.

2) Get metadata associated with the data sets, as this metadata is parameters into the function. FURTHER DISCUSSION: do we want to keep the naming conventions and passed parameters of the function consistent here?? This will definitely simplify things.

3) Get a list of all of the "submitted" modules that we want to run the datasets through. Each module contains the properly labeled function for the associated test (for example, all az-tilt tests require a function called run_az_tilt_estimation()).

4) Run function using the data set and the required metadata parameters and get outputs. Create a dictionary of outputs, containing results, ground truth outputs, and test type.

This is a rough draft that is expected to change.

In [1]:
# Import required packages

import pandas as pd
import glob
from importlib import import_module

In [2]:
# Read in all of the data sets and the associated metadata

data_files = glob.glob("./data/*.csv")
# Drop the metadata file
data_files = [x for x in data_files if 'metadata.csv' not in x]
print(data_files)

# Get the metadata file
metadata_df = pd.read_csv("./data/metadata.csv")

# Get a list of the modules that we're going to be testing. Ideally, this would be housed in some sort of database 
modules_to_import = ["az-tilt-estimator"]

# Generate a list to store all of the results as associated dictionary values
results_list = list()

['./data\\4_ac_power__315.csv']


In [3]:
# Loop through each file, get its associated metadata, and run it through the function
for file in data_files:
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    # Get the metadata associated with the specific file
    associated_metadata = dict(metadata_df[metadata_df['file'] == file.split("\\")[-1]].iloc[0])
    print(associated_metadata)
    # Build pandas series for power stream
    time_series = pd.Series(df[associated_metadata['power_stream']])
    # Import designated modules via importlib
    for mod in modules_to_import:
        mod = import_module(mod)
        function = getattr(mod, "run_az_tilt_estimation")
        tilt, az = function(time_series,
                           associated_metadata['latitude'],
                           associated_metadata['longitude'],
                           associated_metadata['time_zone'])
        print(tilt,az)
        results_list.append({"test": "azimuth-tilt estimation",
                            "data": file,
                            "module": mod,
                            "outputs": (tilt, az),
                            "ground_truth": (associated_metadata['tilt'], associated_metadata['azimuth'])})
    

{'file': '4_ac_power__315.csv', 'power_stream': 'ac_power__315', 'system_id': 4, 'name': 'Sanyo1', 'latitude': 39.7406, 'longitude': -105.1774, 'public': True, 'time_zone': 'Etc/GMT+7', 'azimuth': 180, 'tilt': 40}


C:\Users\kperry\.conda\envs\pvfleets_qa\lib\site-packages\pvanalytics\quality\gaps.py:416: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  mask.loc[start.date():end.date()] = True
C:\Users\kperry\.conda\envs\pvfleets_qa\lib\site-packages\rdtools\filtering.py:641: UserWarning: The XGBoost filter is an experimental clipping filter that is still under development. The API, results, and default behaviors may change in future releases (including MINOR and PATCH). Use at your own risk!
  warnings.warn("The XGBoost filter is an experimental clipping filter "


31.676397658089208 178.6311297551405


In [4]:
print(results_list)

[{'test': 'azimuth-tilt estimation', 'data': './data\\4_ac_power__315.csv', 'module': <module 'az-tilt-estimator' from 'c:\\Users\\kperry\\Documents\\source\\repos\\pvinsight-validation-hub\\az-tilt-estimator.py'>, 'outputs': (31.676397658089208, 178.6311297551405), 'ground_truth': (40, 180)}]
